# 1 Assignment

### Federica Fiorentini, matricola 807124

In [83]:
#Importazione delle librerie necessarie

import numpy as np

import pandas as pd

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE


In [60]:
#Lettura dei file
train = pd.read_csv(r'C:\Users\FedericaFiorentini\Desktop\CreditSafeData\trainCS.csv')
test = pd.read_csv(r'C:\Users\FedericaFiorentini\Desktop\CreditSafeData\testCS.csv')

print("Train shape" + str(train.shape))
print("Test shape" + str(test.shape))

Train shape(8652, 18)
Test shape(10585, 18)


In [61]:
train.head()

,nrOfCompaniesInActivityCode,numberOfEmployees,publicRegisterSection,shareCapital,currentIntRisk (A-E),currentCreditLimit ($),currentRisk (1-100),mainActivityCode,previousIntRisk (A-E),previousCreditLimit ($),returnOnNetAssetsEmployed,activityDays,citizens,AtecoCod,numberOfDirector,companyType,macroRegion,revenue_binned
0,34061.0,NaN,ORDINARIA,100.0,D,0,3.0,3103.0,D,0.0,111.35,43522.0,10977.0,31.0,NaN,srl,Nord - Est,1
1,365109.0,NaN,ORDINARIA,45900.0,B,5000,59.0,682001.0,B,7500.0,-50.86,43474.0,6322.0,68.0,2.0,srl,Nord - Ovest,1
2,694331.0,NaN,IMPRESA ARTIGIANA-ORDINARIA,500.0,D,0,1.0,432101.0,B,2000.0,136.68,43454.0,56962.0,43.0,4.0,srl,Nord - Ovest,1
3,694331.0,2.0,IMPRESA ARTIGIANA-ORDINARIA,10000.0,C,500,49.0,433909.0,C,2500.0,0.00,43452.0,9888.0,43.0,2.0,srl,Nord - Ovest,1
4,398656.0,NaN,ORDINARIA,10000.0,C,500,40.0,412.0,C,2500.0,-23.24,43448.0,214198.0,41.0,1.0,srl,Nord - Est,1


In [62]:
mapping = {'A': 0., 'B':1., 'C':2., 'D':3., 'E': 4.}
train.replace({'currentIntRisk (A-E)' : mapping, 'previousIntRisk (A-E)' : mapping}, regex=True, inplace=True)
test.replace({'currentIntRisk' : mapping, 'previousIntRisk' : mapping}, regex=True, inplace=True)

In [63]:
mapping = {'srl': 0., 'altro':1., 'coop':2.}
train.replace({'companyType' : mapping}, regex=True, inplace=True)
test.replace({'companyType' : mapping}, regex=True, inplace=True)

In [64]:
mapping = {'Nord - Ovest': 0., 'Nord - Est':1., 'Centro':2., 'Sud': 3., 'Isole':4.}
train.replace({'macroRegion' : mapping}, regex=True, inplace=True)
test.replace({'macroRegion' : mapping}, regex=True, inplace=True)

In [65]:
train=train.iloc[:, train.columns!='publicRegisterSection']
test=test.iloc[:, test.columns!='publicRegisterSection']

In [66]:
#verifica dell'assenza di missing value e analisi delle principali statistiche
train.describe()

,nrOfCompaniesInActivityCode,numberOfEmployees,shareCapital,currentIntRisk (A-E),currentCreditLimit ($),currentRisk (1-100),mainActivityCode,previousIntRisk (A-E),previousCreditLimit ($),returnOnNetAssetsEmployed,activityDays,citizens,AtecoCod,numberOfDirector,companyType,macroRegion,revenue_binned
count,8.646000e+03,7970.000000,8.644000e+03,8652.000000,8.652000e+03,8482.000000,8646.000000,8652.000000,8.651000e+03,8626.000000,8117.000000,8.117000e+03,8117.000000,8467.000000,8652.000000,8651.000000,8652.000000
mean,3.858628e+05,10.723839,1.437447e+05,1.530629,2.417880e+04,50.227659,138150.195697,1.559524,2.292538e+04,25.743668,38798.589134,2.592578e+05,43.319576,1.933270,0.139852,2.009363,3.646209
std,3.445221e+05,30.123519,3.617440e+06,0.816323,1.347395e+05,17.064397,221976.085205,0.828234,1.198471e+05,326.447377,2934.919213,6.530960e+05,18.781233,1.630217,0.505750,1.228291,1.650970
min,2.060000e+02,0.000000,-2.178440e+05,0.000000,0.000000e+00,1.000000,1.000000,0.000000,0.000000e+00,-15897.470000,12870.000000,2.160000e+02,1.000000,1.000000,0.000000,0.000000,1.000000
25%,7.065400e+04,2.000000,1.000000e+04,1.000000,2.000000e+03,43.000000,1086.000000,1.000000,2.000000e+03,4.950000,35886.000000,1.095700e+04,33.000000,1.000000,0.000000,1.000000,2.000000
50%,3.651090e+05,5.000000,1.040000e+04,1.000000,7.500000e+03,55.000000,13921.000000,1.000000,7.500000e+03,18.695000,39163.000000,3.134100e+04,43.000000,1.000000,0.000000,2.000000,4.000000
75%,6.943310e+05,10.000000,4.906300e+04,2.000000,2.000000e+04,62.000000,259999.000000,2.000000,2.000000e+04,45.572500,41367.000000,1.159270e+05,47.000000,2.000000,0.000000,3.000000,5.000000
max,1.155302e+06,1212.000000,3.009260e+08,4.000000,7.650000e+06,89.000000,960909.000000,4.000000,5.250000e+06,11852.660000,43683.000000,2.761477e+06,96.000000,26.000000,2.000000,4.000000,6.000000


In [67]:
#Split del dataset in train set e validation set

X=train.loc[:, train.columns!='revenue_binned']
Y=train['revenue_binned']

X_train, X_val, Y_train, Y_val= train_test_split(X,Y, test_size=0.1, random_state=0)

print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(7786, 16)
(866, 16)
(7786,)
(866,)


Per evitare che il peso delle diverse feature all'interno del modello 
sia differente tra loro, si effettua una normalizzazione per riportare i valori delle variabili in un intervallo compreso tra (-1,1). 

Nella normalizzazione, però, vengono escluse la variabile binaria 'Sex' e le categoriche 'Marriage', 'Education' e 'PAY_0'-'PAY_6', in quanto contano solamente rispettivamente 4, 6 e 8 modalità.

In [68]:
X_train.head()

,nrOfCompaniesInActivityCode,numberOfEmployees,shareCapital,currentIntRisk (A-E),currentCreditLimit ($),currentRisk (1-100),mainActivityCode,previousIntRisk (A-E),previousCreditLimit ($),returnOnNetAssetsEmployed,activityDays,citizens,AtecoCod,numberOfDirector,companyType,macroRegion
1437,398656.0,2.0,10000.0,1.0,5000,57.0,412.0,2.0,3000.0,11.47,41472.0,18513.0,41.0,2.0,0.0,0.0
4938,398656.0,7.0,10200.0,1.0,20000,60.0,412.0,1.0,20000.0,3.73,36551.0,1549.0,41.0,2.0,0.0,1.0
5260,398656.0,5.0,10400.0,1.0,7500,55.0,412.0,2.0,3000.0,49.74,35114.0,2761477.0,41.0,1.0,0.0,2.0
6590,83058.0,42.0,4010000.0,1.0,165000,67.0,1072.0,2.0,5000.0,12.37,43258.0,38690.0,10.0,2.0,0.0,3.0
7863,41745.0,15.0,30000.0,1.0,24000,65.0,282.0,1.0,23500.0,36.73,35114.0,29034.0,28.0,2.0,0.0,0.0


In [69]:
X_train.head()

,nrOfCompaniesInActivityCode,numberOfEmployees,shareCapital,currentIntRisk (A-E),currentCreditLimit ($),currentRisk (1-100),mainActivityCode,previousIntRisk (A-E),previousCreditLimit ($),returnOnNetAssetsEmployed,activityDays,citizens,AtecoCod,numberOfDirector,companyType,macroRegion
1437,398656.0,2.0,10000.0,1.0,5000,57.0,412.0,2.0,3000.0,11.47,41472.0,18513.0,41.0,2.0,0.0,0.0
4938,398656.0,7.0,10200.0,1.0,20000,60.0,412.0,1.0,20000.0,3.73,36551.0,1549.0,41.0,2.0,0.0,1.0
5260,398656.0,5.0,10400.0,1.0,7500,55.0,412.0,2.0,3000.0,49.74,35114.0,2761477.0,41.0,1.0,0.0,2.0
6590,83058.0,42.0,4010000.0,1.0,165000,67.0,1072.0,2.0,5000.0,12.37,43258.0,38690.0,10.0,2.0,0.0,3.0
7863,41745.0,15.0,30000.0,1.0,24000,65.0,282.0,1.0,23500.0,36.73,35114.0,29034.0,28.0,2.0,0.0,0.0


In [70]:
#Standardizzazione dei dati

num_X_train = X_train.iloc[:, np.r_[0:2,4,5,9:12,14]]
num_X_val = X_val.iloc[:, np.r_[0:2,4,5,9:12,14]]

num_X_train = num_X_train.astype('float32') 
num_X_val = num_X_val.astype('float32')

scaler = preprocessing.MinMaxScaler((-1,1))
scaler.fit(num_X_train)
scaler.fit(num_X_val)

num_XX_train = scaler.transform(num_X_train.values)
num_XX_val  = scaler.transform(num_X_val.values) 

YY_train = Y_train.values 
YY_val  = Y_val.values 

X_train.iloc[:, np.r_[0:2,4,5,9:12,14]] = num_XX_train
X_val.iloc[:, np.r_[0:2,4,5,9:12,14]] = num_XX_val

XX_train = X_train
XX_val = X_val


print (XX_train.shape, YY_train.shape)
print (XX_val.shape, YY_val.shape)

(7786, 16) (7786,)
(866, 16) (866,)


c:\tools\anaconda3\envs\aml\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [73]:
yy_train = YY_train - 1

array([1, 3, 3, 5, 5], dtype=int64)

In [80]:
yy_val = YY_val - 1

In [74]:
#verifichiamo le frequenze della classe target nel train set e nel validation set in modo tale
#da evitare il class imbalance problem

np.unique(yy_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([ 945, 1276, 1469, 1438, 1175, 1483], dtype=int64))

Le classi all'interno dei due dataset sono sbilanciate in quando la classe negativa ha una frequenza molto più alta, circa del 70% in più rispetto alla classe positiva. Per evitare che questo crei problemi durante lo sviluppo del modello, in quanto si rischia che il modello predica solamente la classe negativa, si procede effettuando un oversampling SMOTE dei dati.

In particolare, è stato scelto di effettuare un oversampling rispetto ad altre tecniche (es. undersampling) per evitare di perdere un grande numero di osservazioni del dataset iniziale. L'obiettivo di questa tecnica è di ottenere un dataset in cui la frequenza delle due classi della variabile target è circa la stessa (18882). 


Si procede con lo sviluppo del modello.

In [90]:
n_epochs = 12
batch_size = 100

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(16,)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='sgd',
              loss='crossentropy',
              metrics=['accuracy'])

ValueError: Unknown loss function:crossentropy

E' stato scelto di sviluppare una rete neurale *Feed-Forward Fully Connected*. In questo particolare tipo di rete, ogni layer necessita di un input, che coincide con l'output del layer precedente, e di una funzione di attivazione. Il modello sopra sviluppato consiste in una rete avente 5 layer, per ogni layer è stata utilizzata come funzione di attivazione la *relu* in quanto risultavano migliori performance, eccetto nell'ultimo strato in cui è stata utilizzata, siccome il problema in questione è un problema di classificazione binaria, la *sigmoide*. 


Inoltre, per l'ottimizzazione del modello, in cui si minimizza la funzione di loss, viene utilizzato il metodo dello *Stochastic Gradient Descent*. 

Per quanto riguarda la loss function è stata utilizzata la *binary crossentropy*, essendo appunto in presenza di un problema di classificazione binaria. L'obiettivo è quello di minimizzare la funzione di perdita durante la stima dei parametri W e b. 

In [88]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 1024)              17408     
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               262400    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 64)               

Per apprendere il modello, si allena la rete sul training set utilizzando dei mini batch di dimensione 100, iterando per 12 epoche.

In [89]:
history = model.fit(XX_train, yy_train, epochs=n_epochs, batch_size=batch_size, validation_data=(XX_val, yy_val)) 
print(history)

Train on 7786 samples, validate on 866 samples
Epoch 1/12
7786/7786 [==============================] - 1s 118us/step - loss: nan - accuracy: 0.1210 - val_loss: nan - val_accuracy: 0.1316
Epoch 2/12
7786/7786 [==============================] - 1s 90us/step - loss: nan - accuracy: 0.1214 - val_loss: nan - val_accuracy: 0.1316
Epoch 3/12
7786/7786 [==============================] - 1s 90us/step - loss: nan - accuracy: 0.1214 - val_loss: nan - val_accuracy: 0.1316
Epoch 4/12
7786/7786 [==============================] - 1s 90us/step - loss: nan - accuracy: 0.1214 - val_loss: nan - val_accuracy: 0.1316
Epoch 5/12
7786/7786 [==============================] - 1s 91us/step - loss: nan - accuracy: 0.1214 - val_loss: nan - val_accuracy: 0.1316
Epoch 6/12
7786/7786 [==============================] - 1s 91us/step - loss: nan - accuracy: 0.1214 - val_loss: nan - val_accuracy: 0.1316
Epoch 7/12
7786/7786 [==============================] - 1s 90us/step - loss: nan - accuracy: 0.1214 - val_loss: nan - 

In [18]:
score = model.evaluate(XX_val, YY_val, batch_size=batch_size) #evaluating the models accuracy or loss,
print('validation loss, validation acc:', score)

2700/2700 [==============================] - 0s 25us/step
validation loss, validation acc: [0.41824995036478396, 0.830740749835968]


In [19]:
predictions = model.predict(XX_val)
y_classes = list(map(int, predictions>0.5))


print(classification_report(YY_val, y_classes))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      2145
           1       0.66      0.37      0.47       555

    accuracy                           0.83      2700
   macro avg       0.76      0.66      0.68      2700
weighted avg       0.81      0.83      0.81      2700



Siccome il dataset iniziale presentava la classe target fortemente sbilanciata tra le due frequenze, è conveniente osservare le misure di Precision e Recall per analizzare le performance del modello sviluppato. In questo caso le performance sembrano essere abbastanza soddisfacenti. 

Per completezza, invece di decidere a priori il numero di epoche in cui in modello viene allenato, si procede implementando un *early stopping* in cui, se la funzione di loss non diminuisce per 10 epoche successive, il modello si blocca. 
Il numero massimo di epoche è pari a 150. 


In [20]:
n_epochs=150
batch_size=100

fBestModel = 'best_model.h5' 
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1) 
best_model = ModelCheckpoint(fBestModel, verbose=0, save_best_only=True)

history = model.fit(XX_train, YY_train, validation_data = (XX_val, YY_val), epochs=n_epochs, 
          batch_size=batch_size, verbose=True, callbacks=[best_model, early_stop]) 

score = model.evaluate(XX_val, YY_val, batch_size=batch_size) #evaluating the models accuracy or loss,
print('validation loss, validation acc:', score)

Train on 24300 samples, validate on 2700 samples
Epoch 1/150
24300/24300 [==============================] - 2s 87us/step - loss: 0.4310 - accuracy: 0.8216 - val_loss: 0.4183 - val_accuracy: 0.8304
Epoch 2/150
24300/24300 [==============================] - 2s 87us/step - loss: 0.4308 - accuracy: 0.8211 - val_loss: 0.4193 - val_accuracy: 0.8289
Epoch 3/150
24300/24300 [==============================] - 2s 87us/step - loss: 0.4303 - accuracy: 0.8227 - val_loss: 0.4166 - val_accuracy: 0.8322
Epoch 4/150
24300/24300 [==============================] - 2s 87us/step - loss: 0.4303 - accuracy: 0.8221 - val_loss: 0.4168 - val_accuracy: 0.8319
Epoch 5/150
24300/24300 [==============================] - 2s 88us/step - loss: 0.4303 - accuracy: 0.8225 - val_loss: 0.4163 - val_accuracy: 0.8319
Epoch 6/150
24300/24300 [==============================] - 2s 88us/step - loss: 0.4291 - accuracy: 0.8222 - val_loss: 0.4166 - val_accuracy: 0.8326
Epoch 7/150
24300/24300 [==============================] - 2s 8

In [54]:
predictions = model.predict(XX_val)
y_classes = list(map(int, predictions>0.5))

print(classification_report(YY_val, y_classes))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      2145
           1       0.67      0.36      0.47       555

    accuracy                           0.83      2700
   macro avg       0.76      0.66      0.69      2700
weighted avg       0.82      0.83      0.81      2700



Il modello viene trainato in 15 epoche e ciò significa che alla 5 epoca ha raggiunto le performance migliori e per 10 epoche successive non è più migliorato. Infatti, il valore dell'accuracy è leggermente migliorato rispetto al modello precedente. 

Per concludere, si applica il modello stimato al test set con l'obiettivo di predire la classe di 3000 osservazioni. 



In [56]:
num_X_test = test.iloc[:,np.r_[0,4, 11:23]]
num_X_test= num_X_test.astype('float32')

scaler_test = preprocessing.MinMaxScaler((-1,1))
scaler_test.fit(num_X_test)

num_XX_test = scaler_test.transform(num_X_test.values)

X_train.iloc[:,np.r_[0,4, 11:23]] = num_XX_train
test.iloc[:,np.r_[0,4, 11:23]] = num_XX_test

XX_test=test
XX_test.head()

c:\tools\anaconda3\envs\aml\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,-0.945206,2,3,1,-0.115385,1,-2,-2,-2,-2,...,-0.905923,-0.885664,-0.876991,-0.879182,-1.000000,-1.000000,-1.000000,-1.000000,-0.998047,-1.000000
1,-0.315068,2,1,2,-0.730769,0,0,0,0,0,...,-0.646854,-0.576258,-0.596392,-0.636422,-0.915114,-0.980623,-0.979426,-0.998409,-0.896545,-0.452668
2,-0.808219,1,1,2,-0.846154,0,0,0,0,0,...,-0.759417,-0.708140,-0.684016,-0.690910,-0.974547,-0.983617,-0.988179,-0.954854,-0.984539,-0.954622
3,-0.835616,2,2,2,-0.192308,1,2,0,0,0,...,-0.742819,-0.726492,-0.710852,-0.715963,-0.999915,-0.989412,-0.987588,-0.978556,-0.990512,-0.982348
4,0.068493,1,3,1,-0.538462,0,0,2,0,0,...,-0.899865,-0.848844,-0.857683,-0.863666,-0.965986,-0.999971,-0.939432,-0.946716,-0.978798,-0.983165


In [57]:
test_predictions = model.predict(XX_test)
y_test_class = list(map(int, test_predictions>0.5))


np.savetxt(r'Desktop/Federica_Fiorentini_807124.txt', y_test_class, fmt='%d')